In [1]:
import modeling
import numpy as np
import json
import tensorflow as tf

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
ALBERT_INIT_CHKPNT = 'pretraining_output2/model.ckpt-1000002'
ALBERT_CONFIG = 'albert_config/albert_config_base.json'

In [3]:
albert_config = modeling.BertConfig.from_json_file(ALBERT_CONFIG)

In [6]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=albert_config,
            is_training=True,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)

In [8]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ALBERT_INIT_CHKPNT)

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


embedding_lookup_factorized. factorized embedding parameterization is used.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from pretraining_output2/model.ckpt-1000002


In [9]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'albert-base/model.ckpt')

'albert-base/model.ckpt'

In [10]:
!cp sp10m.cased.v6.* albert-base
!cp albert_config/albert_config_base.json albert-base

In [11]:
!tar cvzf albert-base-11-10-2019.tar.gz albert-base

albert-base/
albert-base/model.ckpt.index
albert-base/albert_config_base.json
albert-base/sp10m.cased.v6.model
albert-base/model.ckpt.data-00000-of-00001
albert-base/sp10m.cased.v6.vocab
albert-base/checkpoint
albert-base/model.ckpt.meta


In [12]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'albert-base-11-10-2019.tar.gz'
outPutname = "bert-bahasa/albert-base-11-10-2019.tar.gz"

s3 = boto3.client('s3',
                 aws_access_key_id='',
                 aws_secret_access_key='')
s3.upload_file(Key,bucketName,outPutname)